In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Eczema"
cohort = "GSE123086"

# Input paths
in_trait_dir = "../../input/GEO/Eczema"
in_cohort_dir = "../../input/GEO/Eczema/GSE123086"

# Output paths
out_data_file = "../../output/preprocess/Eczema/GSE123086.csv"
out_gene_data_file = "../../output/preprocess/Eczema/gene_data/GSE123086.csv"
out_clinical_data_file = "../../output/preprocess/Eczema/clinical_data/GSE123086.csv"
json_path = "../../output/preprocess/Eczema/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A validated single-cell-based strategy to identify diagnostic and therapeutic targets in complex diseases [study of 13 diseases]"
!Series_summary	"We conducted prospective clinical studies to validate the importance of CD4+ T cells in 13 diseases from the following ICD-10-CM chapters: Neoplasms (breast cancer, chronic lymphocytic leukemia); endocrine, nutritional and metabolic diseases (type I diabetes, obesity); diseases of the circulatory system (atherosclerosis); diseases of the respiratory system (acute tonsillitis, influenza, seasonal allergic rhinitis, asthma); diseases of the digestive system (Crohn’s disease [CD], ulcerative colitis [UC]); and diseases of the skin and subcutaneous tissue (atopic eczema, psoriatic diseases)."
!Series_summary	"Study participants were recruited by clinical specialists based on diagnostic criteria defined by organizations representing each specialist’s discipline. Age and gender matched healthy controls (n = 1

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this study used microarrays to analyze gene expression
# from CD4+ T cells, so gene expression data should be available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (eczema):
# Looking at the sample characteristics, primary diagnosis is at index 1
# and includes "ATOPIC_ECZEMA" as one of the possible values
trait_row = 1

# For age:
# Age information appears to be available at indices 3 and 4
age_row = 3

# For gender:
# Gender information (Sex) appears to be at index 2
gender_row = 2

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert trait value to binary (1 for Eczema, 0 for control)"""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Map values to binary
    if 'ATOPIC_ECZEMA' in value:
        return 1
    elif 'HEALTHY_CONTROL' in value:
        return 0
    return None

def convert_age(value):
    """Convert age value to continuous numeric value"""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to convert to float
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Map values to binary
    if value.upper() == 'FEMALE':
        return 0
    elif value.upper() == 'MALE':
        return 1
    return None

# 3. Save Metadata
# Check if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, 
                             cohort=cohort, 
                             info_path=json_path, 
                             is_gene_available=is_gene_available,
                             is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Use the sample characteristics dictionary to create a DataFrame
    # This dictionary is assumed to be available from the previous step
    sample_chars_dict = {0: ['cell type: CD4+ T cells'], 
                         1: ['primary diagnosis: ASTHMA', 'primary diagnosis: ATHEROSCLEROSIS', 
                             'primary diagnosis: BREAST_CANCER', 'primary diagnosis: CHRONIC_LYMPHOCYTIC_LEUKEMIA', 
                             'primary diagnosis: CROHN_DISEASE', 'primary diagnosis: ATOPIC_ECZEMA', 
                             'primary diagnosis: HEALTHY_CONTROL', 'primary diagnosis: INFLUENZA', 
                             'primary diagnosis: OBESITY', 'primary diagnosis: PSORIASIS', 
                             'primary diagnosis: SEASONAL_ALLERGIC_RHINITIS', 'primary diagnosis: TYPE_1_DIABETES', 
                             'primary diagnosis: ACUTE_TONSILLITIS', 'primary diagnosis: ULCERATIVE_COLITIS'], 
                         2: ['Sex: Male', 'diagnosis2: ATOPIC_ECZEMA', 'Sex: Female', 'diagnosis2: ATHEROSCLEROSIS', 
                             'diagnosis2: ASTHMA_OBESITY', 'diagnosis2: ASTHMA', 
                             'diagnosis2: ASTMHA_SEASONAL_ALLERGIC_RHINITIS', 'diagnosis2: OBESITY'], 
                         3: ['age: 56', 'Sex: Male', 'age: 20', 'age: 51', 'age: 37', 'age: 61', 'age: 31', 
                             'age: 41', 'age: 80', 'age: 53', 'age: 73', 'age: 60', 'age: 76', 'age: 77', 
                             'age: 74', 'age: 69', 'age: 81', 'age: 70', 'age: 82', 'age: 67', 'age: 78', 
                             'age: 72', 'age: 66', 'age: 36', 'age: 45', 'age: 65', 'age: 48', 'age: 50', 
                             'age: 24', 'age: 42'], 
                         4: [float('nan'), 'age: 63', 'age: 74', 'age: 49', 'age: 60', 'age: 68', 'age: 38', 
                             'age: 16', 'age: 12', 'age: 27']}
    
    clinical_data = pd.DataFrame.from_dict(sample_chars_dict, orient='index')
    
    try:
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the data
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    
    except Exception as e:
        print(f"An error occurred during clinical feature extraction: {e}")


Preview of selected clinical features:
{0: [nan, 56.0, 1.0], 1: [nan, nan, nan], 2: [nan, 20.0, 0.0], 3: [nan, 51.0, nan], 4: [nan, 37.0, nan], 5: [1.0, 61.0, nan], 6: [0.0, 31.0, nan], 7: [nan, 41.0, nan], 8: [nan, 80.0, nan], 9: [nan, 53.0, nan], 10: [nan, 73.0, nan], 11: [nan, 60.0, nan], 12: [nan, 76.0, nan], 13: [nan, 77.0, nan], 14: [nan, 74.0, nan], 15: [nan, 69.0, nan], 16: [nan, 81.0, nan], 17: [nan, 70.0, nan], 18: [nan, 82.0, nan], 19: [nan, 67.0, nan], 20: [nan, 78.0, nan], 21: [nan, 72.0, nan], 22: [nan, 66.0, nan], 23: [nan, 36.0, nan], 24: [nan, 45.0, nan], 25: [nan, 65.0, nan], 26: [nan, 48.0, nan], 27: [nan, 50.0, nan], 28: [nan, 24.0, nan], 29: [nan, 42.0, nan]}
Clinical data saved to ../../output/preprocess/Eczema/clinical_data/GSE123086.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Eczema/GSE123086/GSE123086_series_matrix.txt.gz


Gene data shape: (22881, 166)
First 20 gene/probe identifiers:
Index(['1', '2', '3', '9', '10', '12', '13', '14', '15', '16', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers are not human gene symbols. They appear to be numeric probe identifiers 
# from a microarray platform, which need to be mapped to actual gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's explore the SOFT file more thoroughly to find gene symbols
print("\nExploring SOFT file more thoroughly for gene information:")
gene_info_patterns = []
entrez_to_symbol = {}

with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if i < 1000:  # Check header section for platform info
            if '!Series_platform_id' in line or '!Platform_title' in line:
                print(line.strip())
                
        # Look for gene-related columns and patterns in the file
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line or 'Symbol' in line:
            gene_info_patterns.append(line.strip())
            
        # Extract a mapping using ENTREZ_GENE_ID if available
        if len(gene_info_patterns) < 2 and 'ENTREZ_GENE_ID' in line and '\t' in line:
            parts = line.strip().split('\t')
            if len(parts) >= 2:
                try:
                    # Attempt to add to mapping - assuming ENTREZ_GENE_ID could help with lookup
                    entrez_id = parts[1]
                    probe_id = parts[0]
                    if entrez_id.isdigit() and entrez_id != probe_id:
                        entrez_to_symbol[probe_id] = entrez_id
                except:
                    pass
        
        if i > 10000 and len(gene_info_patterns) > 0:  # Limit search but ensure we found something
            break

# Show some of the patterns found
if gene_info_patterns:
    print("\nFound gene-related patterns:")
    for pattern in gene_info_patterns[:5]:
        print(pattern)
else:
    print("\nNo explicit gene info patterns found")

# Let's try to match the ENTREZ_GENE_ID to the probe IDs
print("\nAnalyzing ENTREZ_GENE_ID column:")
if 'ENTREZ_GENE_ID' in gene_annotation.columns:
    # Check if ENTREZ_GENE_ID contains actual Entrez IDs (different from probe IDs)
    gene_annotation['ENTREZ_GENE_ID'] = gene_annotation['ENTREZ_GENE_ID'].astype(str)
    different_ids = (gene_annotation['ENTREZ_GENE_ID'] != gene_annotation['ID']).sum()
    print(f"Number of entries where ENTREZ_GENE_ID differs from ID: {different_ids}")
    
    if different_ids > 0:
        print("Some ENTREZ_GENE_ID values differ from probe IDs - this could be useful for mapping")
        # Show examples of differing values
        diff_examples = gene_annotation[gene_annotation['ENTREZ_GENE_ID'] != gene_annotation['ID']].head(5)
        print(diff_examples)
    else:
        print("ENTREZ_GENE_ID appears to be identical to probe ID - not useful for mapping")

# Search for additional annotation information in the dataset
print("\nLooking for alternative annotation approaches:")
print("- Checking for platform ID or accession number in SOFT file")

platform_id = None
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Platform_geo_accession' in line:
            platform_id = line.split('=')[1].strip().strip('"')
            print(f"Found platform GEO accession: {platform_id}")
            break
        if i > 200:
            break

# If we don't find proper gene symbol mappings, prepare to use the ENTREZ_GENE_ID as is
if 'ENTREZ_GENE_ID' in gene_annotation.columns:
    print("\nPreparing provisional gene mapping using ENTREZ_GENE_ID:")
    mapping_data = gene_annotation[['ID', 'ENTREZ_GENE_ID']].copy()
    mapping_data.rename(columns={'ENTREZ_GENE_ID': 'Gene'}, inplace=True)
    print(f"Provisional mapping data shape: {mapping_data.shape}")
    print(preview_df(mapping_data, n=5))
else:
    print("\nWarning: No suitable mapping column found for gene symbols")



Gene annotation preview:
Columns in gene annotation: ['ID', 'ENTREZ_GENE_ID', 'SPOT_ID']
{'ID': ['1', '2', '3', '9', '10'], 'ENTREZ_GENE_ID': ['1', '2', '3', '9', '10'], 'SPOT_ID': [1.0, 2.0, 3.0, 9.0, 10.0]}

Exploring SOFT file more thoroughly for gene information:
!Series_platform_id = GPL25864
!Platform_title = Agilent-039494 SurePrint G3 Human GE v2 8x60K Microarray 039381 (Entrez Gene ID  version)



No explicit gene info patterns found

Analyzing ENTREZ_GENE_ID column:


Number of entries where ENTREZ_GENE_ID differs from ID: 3798412
Some ENTREZ_GENE_ID values differ from probe IDs - this could be useful for mapping
           ID ENTREZ_GENE_ID  SPOT_ID
24166  ID_REF          VALUE      NaN
24167    3553    15.35998289      NaN
24168    1609    10.05521694      NaN
24169   10112     4.22140954      NaN
24170   57827    8.437124629      NaN

Looking for alternative annotation approaches:
- Checking for platform ID or accession number in SOFT file

Preparing provisional gene mapping using ENTREZ_GENE_ID:


Provisional mapping data shape: (3822578, 2)
{'ID': ['1', '2', '3', '9', '10'], 'Gene': ['1', '2', '3', '9', '10']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the column names for gene identifiers and gene symbols
# From the preview, we saw that ID is the probe identifier and ENTREZ_GENE_ID contains the gene IDs
id_column = 'ID'  # Column with probe identifiers that match gene_data index
gene_column = 'ENTREZ_GENE_ID'  # Column with gene identifiers (Entrez IDs in this case)

# 2. Get gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, id_column, gene_column)
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("First few rows of mapping data:")
print(mapping_df.head())

# Check how many unique probe IDs and gene symbols are in the mapping
unique_probes = mapping_df['ID'].nunique()
unique_genes = mapping_df['Gene'].nunique()
print(f"Number of unique probe IDs: {unique_probes}")
print(f"Number of unique gene symbols: {unique_genes}")

# 3. Apply gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene-level data shape: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])

# Save the processed gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

Mapping dataframe shape: (3822578, 2)
First few rows of mapping data:
   ID Gene
0   1    1
1   2    2
2   3    3
3   9    9
4  10   10


Number of unique probe IDs: 24167
Number of unique gene symbols: 1275651


Gene-level data shape: (0, 166)
First few gene symbols:
Index([], dtype='object', name='Gene')
Gene data saved to ../../output/preprocess/Eczema/gene_data/GSE123086.csv
